# Hybrid Search
Hybrid search integrates both keyword-based search and semantic search in order to combine the strengths of both and provide users with a more comprehensive and efficient search experience. This notebook is an example on how to perform hybrid search with SingleStore's database and notebooks.

## Setup
Let's first download the libraries necessary.

In [ ]:
!pip install matplotlib --quiet
!pip install plotly.express --quiet
!pip install scikit-learn --quiet
!pip install tabulate --quiet
!pip install tiktoken --quiet
!pip install wget --quiet
!pip install openai --quiet

In [3]:
import pandas as pd
import os
import wget
import ast
import json

In [4]:
# Import the library for vectorizing the data (Up to 2 minutes)
!pip install sentence-transformers --quiet
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base')

## Import data from CSV File
This csv file holds the title, summary, and category of approximately 2000 news articles.

In [5]:
# download reviews csv file
cvs_file_path = "https://raw.githubusercontent.com/openai/openai-cookbook/main/examples/data/AG_news_samples.csv"
file_path = "AG_news_samples.csv"

if not os.path.exists(file_path):
    wget.download(cvs_file_path, file_path)
    print("File downloaded successfully.")
else:
    print("File already exists in the local file system.")

File already exists in the local file system.


In [6]:
df = pd.read_csv('AG_news_samples.csv')

In [7]:
df

,title,description,label_int,label
0,World Briefings,BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime M...,1,World
1,Nvidia Puts a Firewall on a Motherboard (PC Wo...,PC World - Upcoming chip set will include buil...,4,Sci/Tech
2,"Olympic joy in Greek, Chinese press",Newspapers in Greece reflect a mixture of exhi...,2,Sports
3,U2 Can iPod with Pictures,"SAN JOSE, Calif. -- Apple Computer (Quote, Cha...",4,Sci/Tech
4,The Dream Factory,"Any product, any shape, any size -- manufactur...",4,Sci/Tech
...,...,...,...,...
1995,You Control: iTunes puts control in OS X menu ...,MacCentral - You Software Inc. announced on Tu...,4,Sci/Tech
1996,Argentina beat Italy for place in football final,Favourites Argentina beat Italy 3-0 this morni...,2,Sports
1997,NCAA case no worry for Spurrier,Shortly after Steve Spurrier arrived at Florid...,2,Sports
1998,Secret Service Busts Cyber Gangs,The US Secret Service Thursday announced arres...,4,Sci/Tech


In [8]:
data = df.values.tolist()

## Set up SingleStore Database

Connect to your SingleStoreDB Cloud workspaces using SQLAlchemy.

In [26]:
from sqlalchemy import *

db_connection = create_engine(connection_url)

Set up your SingleStore Database which will hold your data.

In [27]:
%%sql
-- Create the database
DROP DATABASE IF EXISTS news;
CREATE DATABASE IF NOT EXISTS news;

[]

In [28]:
%%sql
USE news;
-- Create the table
DROP TABLE IF EXISTS news_articles;
CREATE TABLE IF NOT EXISTS news_articles (
    id INT PRIMARY KEY,
    title TEXT,
    description TEXT,
    genre TEXT,
    embedding BLOB,
    FULLTEXT (title, description)
);

[]

### Get embeddings for every row based on the description column.

In [ ]:
descriptions = [row[1] if row[1] is not None else row[1] for row in data]
all_embeddings = model.encode(descriptions)
all_embeddings.shape

### Populate the database

In [29]:
%sql TRUNCATE TABLE news_articles;
statement = f"""
        INSERT INTO news.news_articles (
            id,
            title,
            description,
            genre,
            embedding
        )
        VALUES (
            %s,
            %s,
            %s,
            %s,
            JSON_ARRAY_PACK_F64(%s)
        )
    """
for i in range(0, len(data)):
    try:
        ndarray_to_list = all_embeddings[i].tolist()
        json_vector = json.dumps(ndarray_to_list)
        db_connection.execute(statement, (i, data[i][0], data[i][1], data[i][3], str(json_vector)))
    except Exception as e:
        print("Error inserting row {}: {}".format(i, e))
        continue

## Semantic Search

### Connect to OpenAI

In [30]:
import openai

# models
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"

In [31]:
openai.api_key = 'YOUR_API_KEY'

### Run Semantic Search and get scores

In [40]:
from openai.embeddings_utils import get_embedding
search_query = "Articles about Merck stock"
search_embedding = json.dumps(model.encode(search_query).tolist())

# Get the embedding of the query.
query_embedding_response = get_embedding(search_query, EMBEDDING_MODEL)

# Create the SQL statement.
query_statement = f"""
    SELECT
        title,
        description,
        genre,
        DOT_PRODUCT_F64(JSON_ARRAY_PACK_F64(%s), embedding) AS score
    FROM news.news_articles
    ORDER BY score DESC
    LIMIT 10
    """

# Execute the SQL statement.
results = db_connection.execute(query_statement, (search_embedding,)).fetchall()

Printing the results:

In [41]:
output_list = []

for res in results:
    output_list.append([
        res[0], res[2], res[3]
    ])

In [42]:
from tabulate import tabulate
headers = ["title", "genre", "score"]
print(tabulate(output_list, headers))

title                                                              genre        score
-----------------------------------------------------------------  --------  --------
Merck plunges on report it hid Vioxx risks                         Business  0.612733
Drug Firm Shares in Slump                                          Business  0.526167
Merck Pulls Arthritis Drug Off Market                              Business  0.518859
Merck takes down the Dow                                           Business  0.492008
#39;Best #39; ratings a response to rising costs, safety concerns  Business  0.468971
Washington Post Profit Up on Education,TV                          Business  0.447177
DRUG LIABILITY: Attorneys want to query Merck CEO                  Business  0.445199
Dow Jones to Buy MarketWatch for \$463 Mln                         Business  0.417015
Will This Takeover Plan Turn Into Gold? (The Motley Fool)          Sci/Tech  0.376652
Northrop Third-Quarter Profit Rises                   

## Hybrid Search

This search finds the average of the score gotten from the semantic search and the score gotten from the key-word search and sorts the news articles by this combined score to perform an effective hybrid search.

In [123]:
hyb_query = "Articles about Merck stock"
hyb_embedding = json.dumps(model.encode(hyb_query).tolist())

# Get the embedding of the query.
hyb_embedding_response = get_embedding(hyb_query, EMBEDDING_MODEL)

# Create the SQL statement.
hyb_statement = f"""
    SELECT
        title,
        description,
        genre,
        DOT_PRODUCT_F64(JSON_ARRAY_PACK_F64(%s), embedding) AS semantic_score,
        MATCH(title, description) AGAINST (%s) AS keyword_score,
        (semantic_score + keyword_score) / 2 AS combined_score
    FROM news.news_articles
    ORDER BY combined_score DESC
    LIMIT 10
    """

# Execute the SQL statement.
hyb_results = db_connection.execute(hyb_statement, (hyb_embedding, hyb_query)).fetchall()

In [124]:
hybrid_output_list = []

for res in hyb_results:
    hybrid_output_list.append([
        res[0], res[2], res[5]
    ])

In [125]:
from tabulate import tabulate
headers = ["title", "genre", "score"]
print(tabulate(hybrid_output_list, headers))

title                                                                genre        score
-------------------------------------------------------------------  --------  --------
DRUG LIABILITY: Attorneys want to query Merck CEO                    Business  0.492007
Merck plunges on report it hid Vioxx risks                           Business  0.485847
Merck takes down the Dow                                             Business  0.426202
Merck Pulls Arthritis Drug Off Market                                Business  0.402937
Drug Firm Shares in Slump                                            Business  0.331145
US Stock-Index Futures Decline; Citigroup, GE Slip in Europe         Business  0.237872
#39;Best #39; ratings a response to rising costs, safety concerns    Business  0.234486
Owner of Big Electronic Stock Trading System Is Said to Be for Sale  Business  0.224903
Washington Post Profit Up on Education,TV                            Business  0.223588
Dow Jones to Buy MarketWatch for